In [9]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
base_mask_path = r"E:\2_dataset\2D_petAndMask\mask"
patient_txt = r"E:\2_dataset\YoloDataSet\patient_ids.txt"
img_txt = r"E:\2_dataset\YoloDataSet\img_ids.txt"

In [2]:
# 得到所有带标签的图像的id
# 得到所有病人id
img_list = []
patient_list = []
with open(img_txt,"r") as f:
    imgs = f.readlines()
    
    for i in imgs:
        i = i.replace("\n",".png")
        img_list.append(i)
    f.close()

with open(patient_txt,"r") as f:
    p = f.readlines()
    for i in p:
        i = i.replace("\n","")
        patient_list.append(i)
    f.close()


In [24]:
# 生成病人标签文件夹
labels_parent_path=r"E:\PycharmProjects\pythonProject\labels"
for patient in patient_list:
    patient_path = os.path.join(labels_parent_path,patient)
    if not os.path.exists(patient_path):
        os.mkdir(patient_path)

def bnd_box_to_yolo_line(x_min,x_max,y_min,y_max,img_size):
       
        
        x_center = float((x_min + x_max)) / 2 / img_size[1]
        y_center = float((y_min + y_max)) / 2 / img_size[0]

        w = float((x_max - x_min)) / img_size[1]
        h = float((y_max - y_min)) / img_size[0]

        return x_center, y_center, w, h
for i in img_list:

    # 图片所属病人
    img_patient = i[0:5]
    img_path = os.path.join(base_mask_path,i)
     
    img = cv2.imread(img_path,0)
    i.replace(".png","")
    # 将当前的图片转换成numpy数组
    
    img_arry = np.array(img)
    res = np.where(img_arry > 1)
 
    res = zip(res[0],res[1])
    res = list(res)
    xs = []
    ys = []
    for j in res:
        xs.append(j[0])
        ys.append(j[1])
    x1 = min(xs)
    x2 = max(xs)+1
    y1 = min(ys)
    y2 = max(ys)+1
    x_c,y_c,w,h = bnd_box_to_yolo_line(x1,x2,y1,y2,[128,128])
    label = "0 {} {} {} {}".format(x_c,y_c,w,h)

    label_path = os.path.join(labels_parent_path,img_patient,"{}{}".format(i,".txt"))
    with open(label_path,"w") as f:
        f.write(label)
        f.close()

In [23]:
i_path = r"E:\2_dataset\2D_petAndMask\mask\00001023.png"
# 获取坐标值，image的id
def getBoundings(img_path):
    img_id = img_path.split("\\")[-1].split(".")[0]
    img_array = np.array(cv2.imread(img_path, 0))
    '''
    retval 是存在多少个标签（背景也算）
    labels 背景标识为0，第一个区域标识为1，第二个区域标识为2.。。
    stats 将原图的对应区域的值更改为labels后的转换图
    centroids 连通区域的质心
    此处采用8连通作为评判连通区域的标准
    '''
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(img_array, connectivity=8)
    '''
    area_nums是指存在多少个标签
    '''

    if retval > 1:
        area_nums = retval - 1
        boundingboxs = []
        for i, data in enumerate(stats):
            if i > 0:
                x, y, w, h, o = data
                area = x, y, w, h
                boundingboxs.append(area)

        return boundingboxs, img_id
    else:
        return 0, 0

res, id = getBoundings(i_path)

print(res)


[(87, 67, 1, 1)]


In [34]:
# 将病人id划分成训练集与测试集 txt
from operator import le
import random


with open(patient_txt,"r") as file:
    patients_file = file.readlines()
    file.close()
patients = []
for i in patients_file:
    i = i.replace("\n","")
    patients.append(i)
data_len = [i for i in range(len(patients))]

random.shuffle(data_len)
train_idx = []
val_idx = []
for i,idx in enumerate(data_len):
    if i <=132:
        train_idx.append(patients[data_len[idx]])
    else:
        val_idx.append(patients[data_len[idx]])

print(train_idx)
print(val_idx)
with open("train.txt","w") as f:
    for i in train_idx:
        f.write(i+"\n")
    f.close()
with open("valid.txt","w") as f:
    for i in val_idx:
        f.write(i+"\n")
    f.close()

['00089', '00189', '00124', '00184', '00177', '00151', '00082', '00112', '00164', '00099', '00017', '00014', '00156', '00199', '00159', '00076', '00011', '00195', '00005', '00117', '00007', '00092', '00027', '00001', '00018', '00029', '00083', '00022', '00142', '00016', '00050', '00102', '00200', '00107', '00090', '00167', '00058', '00169', '00181', '00152', '00063', '00144', '00154', '00183', '00094', '00149', '00078', '00193', '00025', '00121', '00179', '00061', '00148', '00105', '00098', '00028', '00008', '00128', '00080', '00032', '00079', '00086', '00139', '00126', '00101', '00046', '00173', '00034', '00122', '00095', '00059', '00066', '00171', '00125', '00045', '00085', '00174', '00111', '00009', '00100', '00103', '00048', '00197', '00138', '00104', '00132', '00162', '00108', '00081', '00006', '00135', '00030', '00155', '00191', '00131', '00097', '00020', '00054', '00024', '00143', '00115', '00077', '00110', '00067', '00074', '00161', '00172', '00182', '00019', '00187', '00071', 